***Participantes (RM - NOME):***<br>
341759 - Davidson de Oliveira Mizael<br>
341778 - Nilo José de Andrade Neto<br>
341490 - Eduardo Nascimento<br>


###**Criar um classificador de sentimento aplicando técnicas de PLN**
---

Utilizando o dataset de revisões de filmes em português [1], criar um classificador de sentimentos que consiga um score na métrica F1 Score superior a 70%.

Devem utilizar uma amostra de 20% e randon_state igual a 42 para testar as implementações e mensurar a métrica F1 Score (usar o parâmetro average = 'weighted') o restante dos dados devem ser utilizados para o treinamento (80%).

Fique a vontade para testar os métodos de pré-processamento, abordagens, algoritmos e bibliotecas, mas explique e justifique suas decisões.
O trabalho poderá ser feito em grupo de até 4 pessoas (mesmo grupo do Startup One).

Separe a implementação do seu modelo campeão junto com a parte de validação/teste de forma que o professor consiga executar todo o pipeline do modelo campeão.

Composição da nota:
- 50% - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, etc.)
- 50% - Baseado na performance obtida com o dataset de teste (conforme recomendação da amostra) no seu modelo campeão e na validação que o professor processar (Métrica F1 Score).

[1] - https://dados-ml-pln.s3-sa-east-1.amazonaws.com/reviews-pt-br.csv

Bom desenvolvimento!

In [1]:
import nltk
import pickle
import spacy #!pip install spacy

import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import f1_score


/tmp/ipykernel_35704/793835239.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [2]:
# CARREGANDO O DATA FRAME
import pandas as pd
df = pd.read_csv('data/reviews-pt-br.csv', encoding='utf-8')

# Façam o download do arquivo e utilizem localmente durante os testes

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44514 entries, 0 to 44513
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   codigo      44514 non-null  int64 
 1   texto       44514 non-null  object
 2   sentimento  44514 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [4]:
# Carrega StopWords
nlp = spacy.load('pt_core_news_sm') #python3 -m spacy download pt_core_news_sm
nltk.download('stopwords')

# stopwords NLTK
stops_nltk = nltk.corpus.stopwords.words('portuguese')

# stopwords SpaCy
stops_spacy = nlp.Defaults.stop_words

# stopwords do SpaCy e NLTK combinadas
stops = list(set(stops_spacy).union(set(stops_nltk)))
len(stops)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/njaneto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


499

In [5]:
# função de lematização dos verbos do documento
def fn_lematiza_verb_texto(texto):
    sent = []
    doc = nlp(texto)
    for word in doc:
        if word.pos_ =='VERB':
            sent.append(word.lemma_)
        else:
            sent.append(word.text)
    return " ".join(sent)

# função para limpar documento
def fn_limpa_texto(texto):
    result = []
    texto = re.findall(r"[a-zA-z]+", texto)
    for w in texto:
        w = w.lower()
        if w in stops:
            continue
        result.append(w)
        
    texto = ' '.join(result)
    return texto



In [6]:
df.dropna(inplace=True)
# limpa texto
df['texto_trat'] = df.texto.progress_apply(fn_limpa_texto)

# aplica a lematização dos verbos no dataframe
df['texto_lemma_verb'] = df.texto_trat.progress_apply(fn_lematiza_verb_texto)

df['sentimento'] = df.sentimento.progress_apply(lambda x: 1 if x == 'pos' else 0)

  0%|          | 0/44514 [00:00<?, ?it/s]

  0%|          | 0/44514 [00:00<?, ?it/s]

  0%|          | 0/44514 [00:00<?, ?it/s]

In [7]:
df.head(20)

,codigo,texto,sentimento,texto_trat,texto_lemma_verb
0,1,Esse bocejo de pia de cozinha de orçamento mui...,0,bocejo pia cozinha or amento filme s feito rei...,bocejo pia cozinhar or amento filme s feito re...
1,2,O Bravo parece indicar que o personagem princi...,0,bravo indicar personagem principal claro coraj...,bravo indicar personagem principal claro coraj...
2,3,"Durante a Guerra pela Independência do Sul, GE...",1,durante guerra independ ncia sul general spank...,durante guerra independ ncia sul general spank...
3,4,É fora de questão que a verdadeira Anna Anders...,1,quest verdadeira anna anderson n princesa anas...,quest verdadeira anna anderson n princesa anas...
4,5,Concordo totalmente com outro dos revisores aq...,0,concordo totalmente outro revisores ficou sati...,concordar totalmente outro revisores ficar sat...
5,6,Obra-prima absoluta de um filme! Boa noite Mr....,1,obra prima absoluta filme noite mr tom rapidam...,obra prima absoluta filme noite mr tom rapidam...
6,7,Embora a palavra megalmania seja muito usada p...,1,palavra megalmania usada descrever gene kelly ...,palavra megalmania usar descrever gene kelly s...
7,8,Esta tem que ser a peça mais incrível de porca...,0,pe incr vel porcaria cinematogr fica j assisti...,pe incr vel porcaria cinematogr ficar j assist...
8,9,Eu suponho que todas as piadas internas são o ...,0,suponho piadas internas s munchies cl ssico cu...,suponho piadas internas s munchies cl ssico cu...
9,10,"Se há um tema deste filme, é que as pessoas po...",1,h tema filme pessoas lidar dificuldades imagin...,h tema filme pessoas lidar dificuldades imagin...


In [8]:
# pré-processamento vetorização e BOW
# Frequência de termos - term frequency–inverse document frequency (TF-IDF)
vetor = TfidfVectorizer(ngram_range=(1,2), use_idf=True) 

In [9]:
# texto tratado e verbos lematizado
vetor.fit(df.texto_lemma_verb)
text_vect = vetor.transform(df.texto_lemma_verb)

In [10]:
# separa as amostras de Treino (80%) e Texte (20%)
X_train,X_test,y_train,y_test = train_test_split(text_vect, df['sentimento'], test_size = 0.2, random_state = 42)
print(text_vect.shape)

(44514, 2498707)


In [11]:
# define e treina o modelo de Logistic Regression
modelo_lr = LogisticRegression(random_state=42)

modelo_lr.fit(X_train, y_train)

# valida a parformance
y_prediction = modelo_lr.predict(X_test)
logistic_regression_accuracy = f1_score(y_prediction, y_test, average='weighted')

In [27]:
# define e treina o modelo de Logistic Regression com penalty
modelo_lrp = LogisticRegression(penalty = 'l2', C = 100, random_state=42, solver="saga", max_iter=1000)

modelo_lrp.fit(X_train, y_train)

# valida a parformance
y_prediction = modelo_lrp.predict(X_test)
logistic_regression_penalty_accuracy = f1_score(y_prediction, y_test, average='weighted')

In [13]:
# define e treina o modelo de classificação Decision Tree Classifier
modelo_dtc = DecisionTreeClassifier(random_state=42)

modelo_dtc.fit(X_train, y_train)

# valida a parformance
y_prediction = modelo_dtc.predict(X_test)
decision_tree_accuracy = f1_score(y_prediction, y_test, average='weighted')

In [14]:
# define e treina o modelo de Multinomial
modelo_m = MultinomialNB()

modelo_m.fit(X_train, y_train)

# valida a parformance
y_prediction = modelo_m.predict(X_test)
multinomial_accuracy = f1_score(y_prediction, y_test, average='weighted')

In [28]:
print(f'{logistic_regression_accuracy=}')
print(f'{logistic_regression_penalty_accuracy=}')
print(f'{decision_tree_accuracy=}')
print(f'{multinomial_accuracy=}')

logistic_regression_accuracy=0.8781874371484801
logistic_regression_penalty_accuracy=0.8908354096226934
decision_tree_accuracy=0.7227919173532654
multinomial_accuracy=0.8800462145834034


In [29]:
pickle.dump(vetor, open('modelo/modelo_verorizacao.pkl', 'wb'))
pickle.dump(modelo_lrp, open('modelo/modelo_classificacao.pkl', 'wb'))

####**Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.

In [35]:
novo_vetor = pickle.load(open('modelo/modelo_verorizacao.pkl', 'rb'))
novo_modelo = pickle.load(open('modelo/modelo_classificacao.pkl', 'rb'))

In [31]:
dfValida = df.sample(frac=0.25)
dfValida.info()
dfValida.sentimento.value_counts()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128 entries, 36447 to 4725
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   codigo            11128 non-null  int64 
 1   texto             11128 non-null  object
 2   sentimento        11128 non-null  int64 
 3   texto_trat        11128 non-null  object
 4   texto_lemma_verb  11128 non-null  object
dtypes: int64(2), object(3)
memory usage: 521.6+ KB


0    5581
1    5547
Name: sentimento, dtype: int64

In [32]:
text_vect = novo_vetor.transform(dfValida.texto_lemma_verb)

In [33]:
predito = novo_modelo.predict(text_vect)

In [34]:
# valida a parformance
accuracy = f1_score(predito, dfValida.sentimento, average='weighted')

print(text_vect.shape)
print(accuracy)

(11128, 2498707)
0.9787921864780645
